In this notebook we will discuss BiLSTM based pair wise ranking approach.

We divide this into following major phases 

* **Data pre-processing** where the objective is to create (question,correct path, incorrect path) tuples.
    * Creating core chain to their surface forms equivalent.
    * Converting all the paths and question to an intenger space.
    * Creating a tuple of (question, correct path, incorrect path) --> all in integer space. 
* **Neural Model Definition** where we intent to create a model which can take question and core chains and return two encoded vectors.
* **Training Loop**

In [1]:
import json
import time
import traceback
import numpy as np
from tqdm import tqdm
from utils import tensor_utils as tu
from utils import natural_language_utilities as nlutils

#Torch related functionalities
import torch
import torch.nn as nn
import torch.nn.functional as F


np.random.seed(42)

In [2]:
# Loading the dataset
dataset = json.load(open('resources/dataset_with_paths.json'))

# Model

# Data Preprocessing

### 1. Convert URIs to their surface forms in the core chains

['+', 'http://dbpedia.org/ontology/office', '-' , 'http://dbpedia.org/ontology/presidentOf'] -> **`+ office - president of`**


In [3]:
# Snippet for finding the surface forms of the hops in the dataset and adding it to the dataset.
for index,d in tqdm(enumerate(dataset)):
    hop1_sf = []
    hop2_sf = []
    
    for hop1 in d['hop1']:
        # ['+', 'http://dbpedia.org/ontology/office']
        sf1 = nlutils.get_label_via_parsing(hop1[1],lower=True)
        hop1_sf.append([hop1[0],sf1])
    
    for hop2 in d['hop2']:
        # ['+', 'http://dbpedia.org/ontology/office', '-', 'http://dbpedia.org/property/isCitedBy']
        sf1 = nlutils.get_label_via_parsing(hop2[1],lower=True)
        sf2 = nlutils.get_label_via_parsing(hop2[3],lower=True)
        hop2_sf.append([hop2[0],sf1,hop2[2],sf2])
    
    dataset[index]['hop1_sf'] = hop1_sf
    dataset[index]['hop2_sf'] = hop2_sf
    
    # Convert Positive paths as well
    dataset[index]['path_sf'] = [d['path'][0]]
    dataset[index]['path_sf'].append(nlutils.get_label_via_parsing(d['path'][1],lower=True))
    if len(d['path']) > 2:
        dataset[index]['path_sf'].append(d['path'][2])
        dataset[index]['path_sf'].append(nlutils.get_label_via_parsing(d['path'][3],lower=True))

2000it [00:37, 52.68it/s]


### 2. Create a vocabulary 

In [4]:
vocab = {'_pad_':0, '_unk_':1, '+':2, '-':3, '/':4, 'uri':5, 'x':6}

In [5]:
for d in tqdm(dataset):
    
    # Get words from the question
    for token in d['node']['corrected_question'].split():
        vocab.setdefault(token.lower(), len(vocab))
        
    # Get words from the positive path
    for token in d['path_sf']:
        for word in token.split(" "):
            vocab.setdefault(word.lower(), len(vocab))
        
    # Get words from the negative paths
    for path in d['hop1_sf'] + d['hop2_sf']:
        for token in path:
            for word in token.split(" "):
                vocab.setdefault(word.lower(), len(vocab))

itos = {value:key for key,value in vocab.items()}

100%|██████████| 2000/2000 [00:03<00:00, 490.02it/s]


In [6]:
len(itos)

10279

### 3. Create Trainable Data
**Q, P, N** pairs. e.g.

`Who is the president of USA`; `+ president`; `+ capital
 Who is the president of USA`; `+ president`; `- birthplace`
 
 ...

#### Step 3.1 idfy everything. 

e.g. Who is the president of USA -> 13, 24, 39, 21, 77

In [7]:
for index,d in tqdm(enumerate(dataset)):
    
    # Idfyng question
    question_id = [vocab[word.lower()] for word in d['node']['corrected_question'].split()]
    
    # Idfying the positive path
    positive_path_id = []
    for token in d['path_sf']:
        for word in token.split(" "):
            positive_path_id.append(vocab[word.lower()])
    
    #idfying the generated negative path
    negative_paths_id = []
    
    for path in d['hop1_sf'] + d['hop2_sf']:
        path_id = []
        for tokens in path:
            for word in tokens.split(" "):
                path_id.append(vocab[word.lower()])
        negative_paths_id.append(path_id)
    
    dataset[index]['question_id'] = question_id
    dataset[index]['negative_paths_id'] = negative_paths_id
    dataset[index]['positive_path_id'] = positive_path_id

2000it [00:04, 490.24it/s]


#### Step 3.2 split the data in train and test split

In [8]:
train_data,test_data = dataset[:int(len(dataset)*.80)],dataset[int(len(dataset)*.80):] 

#### Step 3.3 create triples --> Question,correct_path,incorrect_path 

In [9]:
train_question = []
train_positive_path = []
train_negative_path = []

for d in train_data:
    for path in d['negative_paths_id']:
        train_question.append(d['question_id'])
        train_positive_path.append(d['positive_path_id'])
        train_negative_path.append(path)

print(f"length of question, positive path and negative paths respecitvely are "
      f"{len(train_question)}, {len(train_positive_path)}, {len(train_negative_path)}")

# @TODO: Shuffle the dataset (train.)
index = np.arange(0,len(train_question))
np.random.shuffle(index)

# randomize
train_question = [train_question[i] for i in index]
train_positive_path = [train_positive_path[i] for i in index]
train_negative_path = [train_negative_path[i] for i in index]

length of question, positive path and negative paths respecitvely are 1147053, 1147053, 1147053


# Neural Network Model

### Recurrent Networks

Language (Sentence) is sequential.

We need a network which can process the inputs one word at a time, where the processing of future words depends upon the words already seen.


We use recurrent networks for this purposes. But this isn't the only form of NNs which can work.

![recurrent nets](img/rnnss.png)

We use $h_6$, or the final output of the RNN as a vector _summarizing_ the sequence, and its encoded representation $\vec{q}$ or $\vec{c}$


In this example, we use the `NotSuchABetterEncoder` class as the RNN, as defined below

In [17]:
class NotSuchABetterEncoder(nn.Module):
    def __init__(self, max_length, hidden_dim, number_of_layer,
                 embedding_dim, vocab_size, bidirectional, device, 
                 dropout=0.0, mode='LSTM', enable_layer_norm=False,
                 vectors=None, debug=False, residual=False):
        '''
            :param max_length: Max length of the sequence.
            :param hidden_dim: dimension of the output of the LSTM.
            :param number_of_layer: Number of LSTM to be stacked.
            :param embedding_dim: The output dimension of the embedding layer/ important only if vectors=none
            :param vocab_size: Size of vocab / number of rows in embedding matrix
            :param bidirectional: boolean - if true creates BIdir LStm
            :param vectors: embedding matrix
            :param debug: Bool/ prints shapes and some other meta data.
            :param enable_layer_norm: Bool/ layer normalization.
            :param mode: LSTM/GRU.
            :param residual: Bool/ return embedded state of the input.
        '''
        super(NotSuchABetterEncoder, self).__init__()

        self.max_length, self.hidden_dim, self.embedding_dim, self.vocab_size = int(max_length), int(hidden_dim), int(embedding_dim), int(vocab_size)
        self.enable_layer_norm = enable_layer_norm
        self.number_of_layer = number_of_layer
        self.bidirectional = bidirectional
        self.dropout = dropout
        self.debug = debug
        self.mode = mode
        self.residual = residual


        assert self.mode in ['LSTM', 'GRU']

        if vectors is not None:
            self.embedding_layer = nn.Embedding.from_pretrained(torch.FloatTensor(vectors))
            self.embedding_layer.weight.requires_grad = True
        else:
            # Embedding layer
            self.embedding_layer = nn.Embedding(self.vocab_size, self.embedding_dim)

        # Mode
        if self.mode == 'LSTM':
            self.rnn = torch.nn.LSTM(input_size=self.embedding_dim,
                                     hidden_size=self.hidden_dim,
                                     num_layers=1,
                                     bidirectional=self.bidirectional)
        elif self.mode == 'GRU':
            self.rnn = torch.nn.GRU(input_size=self.embedding_dim,
                                    hidden_size=self.hidden_dim,
                                    num_layers=1,
                                    bidirectional=self.bidirectional)
        self.dropout = torch.nn.Dropout(p=self.dropout)
        self.device = device
        self.reset_parameters()

    def init_hidden(self, batch_size, device):
        """
            Hidden states to be put in the model as needed.
        :param batch_size: desired batchsize for the hidden
        :param device: torch device
        :return:
        """
        if self.mode == 'LSTM':
            return (torch.ones((1+self.bidirectional , batch_size, self.hidden_dim), device=device),
                    torch.ones((1+self.bidirectional, batch_size, self.hidden_dim), device=device))
        else:
            return torch.ones((1+self.bidirectional, batch_size, self.hidden_dim), device=device)

    def reset_parameters(self):
        """
        Here we reproduce Keras default initialization weights to initialize Embeddings/LSTM weights
        """
        ih = (param for name, param in self.named_parameters() if 'weight_ih' in name)
        hh = (param for name, param in self.named_parameters() if 'weight_hh' in name)
        b = (param for name, param in self.named_parameters() if 'bias' in name)
        for t in ih:
            torch.nn.init.xavier_uniform_(t)
        for t in hh:
            torch.nn.init.orthogonal_(t)
        for t in b:
            torch.nn.init.constant_(t, 0)

    def forward(self, x):
        """

        :param x: input (batch, seq)
        :param h: hiddenstate (depends on mode. see init hidden)
        :param device: torch device
        :return: depends on booleans passed @ init.
        """
        
        # trimming data to remove extra trailing zeros (id for padding)
        x = tu.trim(x)
        
        # Initialize a new hidden state
        h = self.init_hidden(x.shape[0], self.device)

        if self.debug:
            print ("\tx:\t", x.shape)
            if self.mode is "LSTM":
                print ("\th[0]:\t", h[0].shape)
            else:
                print ("\th:\t", h.shape)

        mask = tu.compute_mask(x)

        x = self.embedding_layer(x).transpose(0, 1)

        if self.debug: print ("x_emb:\t\t", x.shape)

        if self.enable_layer_norm:
            seq_len, batch, input_size = x.shape
            x = x.view(-1, input_size)
            x = self.layer_norm(x)
            x = x.view(seq_len, batch, input_size)

        if self.debug: print("x_emb bn:\t", x.shape)

        # get sorted v
        lengths = mask.eq(1).long().sum(1)
        lengths_sort, idx_sort = torch.sort(lengths, dim=0, descending=True)
        _, idx_unsort = torch.sort(idx_sort, dim=0)

        x_sort = x.index_select(1, idx_sort)
        h_sort = (h[0].index_select(1, idx_sort), h[1].index_select(1, idx_sort)) \
            if self.mode is "LSTM" else h.index_select(1, idx_sort)

        x_pack = torch.nn.utils.rnn.pack_padded_sequence(x_sort, lengths_sort)
        x_dropout = self.dropout.forward(x_pack.data)
        x_pack_dropout = torch.nn.utils.rnn.PackedSequence(x_dropout, x_pack.batch_sizes)

        if self.debug:
            print("\nidx_sort:", idx_sort.shape)
            print("idx_unsort:", idx_unsort.shape)
            print("x_sort:", x_sort.shape)
            if self.mode is "LSTM":
                print ("h_sort[0]:\t\t", h_sort[0].shape)
            else:
                print ("h_sort:\t\t", h_sort.shape)


        o_pack_dropout, h_sort = self.rnn.forward(x_pack_dropout, h_sort)
        o, _ = torch.nn.utils.rnn.pad_packed_sequence(o_pack_dropout)

        # Unsort o based ont the unsort index we made
        o_unsort = o.index_select(1, idx_unsort)  # Note that here first dim is seq_len
        h_unsort = (h_sort[0].index_select(1, idx_unsort), h_sort[1].index_select(1, idx_unsort)) \
            if self.mode is "LSTM" else h_sort.index_select(1, idx_unsort)


        # @TODO: Do we also unsort h? Does h not change based on the sort?

        if self.debug:
            if self.mode is "LSTM":
                print("h_sort\t\t", h_sort[0].shape)
            else:
                print("h_sort\t\t", h_sort.shape)
            print("o_unsort\t\t", o_unsort.shape)
            if self.mode is "LSTM":
                print("h_unsort\t\t", h_unsort[0].shape)
            else:
                print("h_unsort\t\t", h_unsort.shape)

        len_idx = (lengths - 1).view(-1, 1).expand(-1, o_unsort.size(2)).unsqueeze(0)

        if self.debug:
            print("len_idx:\t", len_idx.shape)

        # Need to also return the last embedded state. Wtf. How?

        if self.residual:
            len_idx = (lengths - 1).view(-1, 1).expand(-1, x.size(2)).unsqueeze(0)
            x_last = x.gather(0, len_idx)
            x_last = x_last.squeeze(0)
            return o_unsort, h_unsort[0].transpose(1,0).contiguous().view(h_unsort[0].shape[1], -1) , h_unsort, mask, x, x_last
        else:
            return o_unsort, h_unsort[0].transpose(1,0).contiguous().view(h_unsort[0].shape[1], -1) , h_unsort, mask

#### Encoder Usage

In [18]:
## Initialing the encoder.
vocab_size = 1000
device = torch.device('cpu')

encoder = NotSuchABetterEncoder(
            number_of_layer=1,
            bidirectional=True,
            embedding_dim=300,
            max_length = 25,
            hidden_dim=7,
            vocab_size=vocab_size,
            dropout=0.5,
            vectors=None,
            enable_layer_norm=False,
            mode = 'LSTM',
            device = device,
            debug = False).to(device)


## Fake data to pass through it.
BS = 10 # batch size
SL = 4 # maximum sequence length

sequence_batch = torch.randint(0,vocab_size,(BS,SL), dtype=torch.long)

print(sequence_batch)
print(f"Here, {sequence_batch[0]} is a question, and there are {BS} different questions.")

tensor([[434, 212, 503, 542],
        [470, 599, 347, 545],
        [569, 202,  43, 922],
        [ 84,  14, 981, 656],
        [ 95,  46, 356, 956],
        [416, 102, 908, 402],
        [692,  51, 307,   3],
        [189, 693, 706, 204],
        [419, 297, 709, 392],
        [503, 514, 956, 570]])
Here, tensor([434, 212, 503, 542]) is a question, and there are 10 different questions.


In [19]:
# Passing the data to the model
_, sequence_batch_encoded, _, _ = encoder(sequence_batch)
print(sequence_batch_encoded, sequence_batch_encoded.shape)

tensor([[-0.0331,  0.0467, -0.5595,  0.3182,  0.1419, -0.3353, -0.4046, -0.0354,
         -0.0706, -0.0597, -0.0012, -0.2459, -0.0696, -0.1162],
        [-0.3064, -0.2185,  0.0032,  0.2066, -0.3276,  0.1606, -0.2801, -0.3562,
         -0.0260,  0.0333, -0.0269,  0.1715,  0.3174,  0.5970],
        [-0.2329,  0.4847, -0.1218, -0.5392, -0.0248,  0.0465, -0.1798,  0.1852,
          0.6057,  0.1037, -0.0011,  0.0399,  0.2826, -0.1158],
        [-0.0422,  0.1353, -0.4303,  0.1670,  0.1181, -0.0620,  0.0157, -0.0447,
          0.0924,  0.3610,  0.1419, -0.1265, -0.0204, -0.0568],
        [-0.0652,  0.0067,  0.2815,  0.0066, -0.0535, -0.1491,  0.1591,  0.2933,
          0.0441, -0.5116,  0.6158, -0.0267,  0.0209,  0.6315],
        [-0.1371, -0.1831, -0.2718, -0.0685,  0.2872, -0.2326,  0.1842,  0.7259,
         -0.0537, -0.2072, -0.5387, -0.0334, -0.4122,  0.1925],
        [ 0.3835, -0.3390,  0.3613,  0.0638, -0.3298, -0.0330, -0.3760, -0.0386,
         -0.0575,  0.3747,  0.0124, -0.4666, -0.1

#### Ranking Architecture using RNNs
Here the RNN acts as the Encoder

![ranking architecture](img/archi.png)

In [20]:
class Ranker():

    def __init__(self, _parameter_dict, _device, _debug=False):

        self.debug = _debug
        self.parameter_dict = _parameter_dict
        self.device = _device

        if self.debug:
            print("Init Encoder")

        self.encoder = NotSuchABetterEncoder(
            number_of_layer=1,
            bidirectional=self.parameter_dict['bidirectional'],
            embedding_dim=self.parameter_dict['embedding_dim'],
            max_length = self.parameter_dict['max_length'],
            hidden_dim=self.parameter_dict['hidden_size'],
            vocab_size=self.parameter_dict['vocab_size'],
            dropout=self.parameter_dict['dropout'],
            vectors=self.parameter_dict['vectors'],
            enable_layer_norm=False,
            mode = 'LSTM',
            device = _device,
            debug = self.debug).to(self.device)
        

    def train(self, data):
        '''
            Given data, passes it through model, inited in constructor, returns loss and updates the weight
            :params data: {batch of question, pos paths, neg paths and dummy y labels}
            :params optimizer: torch.optim object
            :params loss fn: torch.nn loss object
            :params device: torch.device object

            returns loss
        '''

        # Unpacking the data and model from args
        ques_batch, pos_batch, neg_batch = data['ques_batch'], data['pos_batch'], data['neg_batch']
        
        # TODO: CAN WE DO WITHOUT THIS?
        neg_batch = tu.no_one_left_behind(neg_batch)

        """
            Encoding all the data
        """    
            
        # Encoding question,positive_path and negative path
        _, ques_batch_encoded, _, _ = self.encoder(ques_batch)
        _, pos_batch_encoded, _, _ = self.encoder(pos_batch)
        _, neg_batch_encoded, _, _  = self.encoder(neg_batch)

        # Calculating dot product
        pos_scores = torch.sum(ques_batch_encoded * pos_batch_encoded, -1)
        neg_scores = torch.sum(ques_batch_encoded * neg_batch_encoded, -1)

        return pos_scores, neg_scores

    def predict(self, ques, paths):
        """
            prediction function.
        """
        with torch.no_grad():
            
            # Set mode to evaluate
            self.encoder.eval()

            _, question, _, _ = self.encoder(ques.long())
            _, paths, _, _ = self.encoder(paths.long())

            score = torch.sum(question * paths, -1)

            # Set mode back to train
            self.encoder.train()
            
            return score

In [25]:
if True:
    parameter_dict = {}
    parameter_dict['bidirectional'] = True
    parameter_dict['embedding_dim'] = 300
    parameter_dict['max_length'] = 25
    parameter_dict['hidden_size'] = 256
    parameter_dict['vocab_size'] = 1000
    parameter_dict['dropout'] = 0.3
    parameter_dict['vectors'] = None
    model = Ranker(parameter_dict, torch.device('cpu'), _debug=False)


    # Data making 
    BS = 64 # Batch Size
    SL = 25 # Maximum Sequence Length and also the sequence length of all tensor.
    data = {}
    data['ques_batch'] = torch.randint(0,999, (BS,SL), dtype=torch.long)
    data['pos_batch'] = torch.randint(0,999, (BS,SL), dtype=torch.long)
    data['neg_batch'] = torch.randint(0,999, (BS,SL), dtype=torch.long)

    # Setting up optimizer and loss function
    optimizer = torch.optim.Adam(list(filter(lambda p: p.requires_grad, model.encoder.parameters())))
    loss_fn = nn.MarginRankingLoss(margin=1,size_average=False)
    
    # Cleanse the gradients (if any) from the model parameters
    optimizer.zero_grad()

    # Passing it through models forward, train in this case.
    pos_scores, neg_scores = model.train(data)
    
    """
        **Explanation of y labels**
        If `y == 1` then it assumed the first input should be ranked higher 
            (have a larger value) than the second input, 
            and vice-versa for `y == -1`. 
            
        We want the pos scores to be higher than neg scores, and thus our Y label should always be 1.
    """
    y_label = torch.ones(BS)
    
    # Compute Loss
    loss = loss_fn(pos_scores, neg_scores, y_label)
    
    # Compute gradients to update parameters
    loss.backward()
    
    # Use the gradients to update model parameters
    optimizer.step()
    
    # Printing the output 
    print(loss)

tensor(85.7175, grad_fn=<SumBackward0>)


#### Sampler

Create a mechanism to iterate over data, one batch at at time


![sampler](https://media.giphy.com/media/H42kukGc8I1IlrC1fg/giphy.gif)

In [26]:
class SimplestSampler:
    """
        Given X and Y matrices (or lists of lists),
            it returns a batch worth of stuff upon __next__
    """

    def __init__(self, data, bs: int = 64):
        '''
            Data has question, positive_path and negative paths as fields.
        '''
        self.question = data["question"]
        self.posp = data["positive_path"]
        self.negp = data["negative_path"]
        self.n = len(self.question)
        self.bs = bs  # Batch Size

    def __len__(self):
        return self.n // self.bs - (1 if self.n % self.bs else 0)

    def __iter__(self):
        self.i, self.iter = 0, 0
        return self

    def __next__(self):
        if self.i + self.bs >= self.n:
            raise StopIteration

        _q, _p, _n = self.question[self.i:self.i + self.bs], \
        self.posp[self.i:self.i + self.bs], self.negp[self.i:self.i + self.bs]
        self.i += self.bs
        return _q, _p, _n

In [27]:
# testing the sampler
data = {
    "question":train_question,
    "positive_path": train_positive_path,
    "negative_path": train_negative_path
}
ss = SimplestSampler(data,4000)

for i,j,k in ss:
    print(len(i),len(j),len(k))
    break

4000 4000 4000


# Training Stuff

#### Data Padding

Questions, and core chains are not of the same length.
E.g.

```
Q. Who is the chancellor of Germany?          (6 words)
Q. Who is the mother     of Jon      Snow?    (7 words)
```

So when the RNN processes (and it does so one batch at a time), it'll stop at the shortest length question. I.e. "snow" will be ignored in our example.

We thus pad the questions to be of the same length, in every batch.

![unpadded batch](img/unpadded.png)

turns to 

![padded batch](img/padded.png)

In [28]:
def pad(data_list, max_length = 25):
    
    padded_data = np.zeros((len(data_list),max_length))
    
    for ind,d in enumerate(data_list):
        padded_data[ind][:min(len(padded_data[ind]),len(d))] = d[:min(len(d),len(padded_data[ind]))]

    return np.reshape(padded_data, (-1,max_length))

### Putting it all together

![summing it all together](img/summing.png)

In [39]:
def training_loop(parameter_dict, data_sampler, ranker, optimizer, loss_func, test_data, device):
    '''
        parameter_dict['epochs'] = 10
        # @TODO: Do we need a function or could it be simplified.
    
    '''
    for epoch in range(parameter_dict['epochs']):
        
        print("Epoch: ", epoch, "/", parameter_dict['epochs'])
        
        epoch_loss = []
        epoch_time = time.time()
        i_batch = 0
        
        for q_b, pp_b, np_b in data_sampler:
            
            batch_time = time.time()
            optimizer.zero_grad()
            
            # Step 1- Input data preparation.
            
            # Step 1a- Pad Data.
            q_b = pad(q_b,parameter_dict['max_length'])
            pp_b = pad(pp_b,parameter_dict['max_length'])
            np_b = pad(np_b,parameter_dict['max_length'])
            
            # Step 1b- Convert data to Torch Tensors.
            ques_batch = torch.tensor(q_b, dtype=torch.long, device=device)
            pos_batch = torch.tensor(pp_b, dtype=torch.long, device=device)
            neg_batch = torch.tensor(np_b, dtype=torch.long, device=device)
            
            # Step 2- Input data label.
            y_label = torch.ones(q_b.shape[0], device=device)
            
            # Step 3- Model (already defined.)
            
            # Step 4- Model output (pass data through model.)
            data_batch = {
                            'ques_batch': ques_batch,
                            'pos_batch': pos_batch,
                            'neg_batch': neg_batch,
            }
            
            pos_scores, neg_scores = ranker.train(data=data_batch)
            
            # Step 5- Compute Loss.
            loss = loss_fn(pos_scores, neg_scores, y_label)

            
            # Step 6- Computing gradient.
            loss.backward()

            # Step 7- Parameter Update.
            optimizer.step()
            
            epoch_loss.append(loss.item())
            
            print("Batch:\t%d" % i_batch, "/%d\t: " % (parameter_dict['batch_size']),
                      "%s" % (time.time() - batch_time),
                      "\t%s" % (time.time() - epoch_time),
                      "\t%s" % (str(loss.item())),
                      end=None if i_batch + 1 == int(int(i_batch) / parameter_dict['batch_size']) else "\n")
            
            i_batch = i_batch + 1
        
        accuracy = evaluation(test_data, ranker, parameter_dict['max_length'], device)
        
        # TODO better printing here.
        print(accuracy)

In [40]:
def evaluation(data, modeler, max_length, device):
    # We will follow the same procedure as that of training data preparation
    
    precision = []
    
    for d in data:
        ques, posp, negp = d['question_id'], d['positive_path_id'], d['negative_paths_id']
        
        # padding the data
        negp_p = pad(negp,max_length)
        posp_p = pad([posp],max_length)
        ques_p = pad([ques],max_length)
        
        # stack negative path and positive path.
        # The reason to stack them is that the model while predicting expects just a set of paths.
        paths = np.vstack((negp_p,posp_p))
        
        # Repeating question "path" number of times.
        ques_p = np.repeat(ques_p, len(paths), axis=0)
        
        # Converting them into pytorch tensor.
        ques_p = torch.tensor(ques_p, dtype=torch.long, device=device)
        paths = torch.tensor(paths, dtype=torch.long, device=device)
        
        # Passing the data through predict function. 
        score = modeler.predict(ques_p, paths)
        
        # Find the index of highest scoring question-corechain (path) pair.
        arg_max = torch.argmax(score)
        
        if arg_max.item() == len(paths)-1:
            precision.append(1)
        else:
            precision.append(0)
            
    print(f"the current precision of the system is, {np.average(precision)}")
        
  

In [ ]:

parameter_dict = {}
parameter_dict['bidirectional'] = True
parameter_dict['embedding_dim'] = 300
parameter_dict['max_length'] = 25
parameter_dict['hidden_size'] = 256
parameter_dict['vocab_size'] = len(vocab)
parameter_dict['dropout'] = 0.3
parameter_dict['vectors'] = None
parameter_dict['epochs'] = 5
parameter_dict['batch_size'] = 4000

device = torch.device('cpu')
model = Ranker(parameter_dict, device, _debug=False)
    
# Setting up optimizer and loss function
optimizer = torch.optim.Adam(list(filter(lambda p: p.requires_grad, model.encoder.parameters())))
loss_fn = nn.MarginRankingLoss(margin=1,size_average=False)

# Sampler
data = {
    "question":train_question[:5000],
    "positive_path": train_positive_path[:5000],
    "negative_path": train_negative_path[:5000]
}

ss = SimplestSampler(data,parameter_dict['batch_size'])


# Training
training_loop(parameter_dict=parameter_dict, data_sampler=ss, ranker=model, 
              optimizer=optimizer, loss_func=loss_fn, test_data=test_data,device = device)

Epoch:  0 / 5
Batch:	0 /4000	:  5.25373911857605 	5.253835439682007 	3384.860107421875


In [36]:
test_data[0].keys()

dict_keys(['node', 'parsed_sparql', 'path', 'entity', 'constraints', 'updated_sparql', 'hop1', 'hop2', 'error_flag', 'rdf_constraint', 'hop1_sf', 'hop2_sf', 'path_sf', 'question_id', 'negative_paths_id', 'positive_path_id'])